In [12]:
include("state_utils.jl")

load_patients (generic function with 1 method)

In [13]:
state_vectors_file = apath * "state_vectors.json" 
C = restore_state_vectors(state_vectors_file)

49×750 Array{Float64,2}:
  0.40622    -0.359482    0.355052    0.421069  …   0.299349     -0.0744802
 -0.403457    0.227875    0.135838   -0.337671     -0.295816      0.0895853
  0.057937   -0.0427263  -0.0962289   0.424601      0.254819      0.189519
  0.323062    0.553472   -0.39423     0.918227      0.206701      0.421109
  0.262369    0.770682   -0.353434    0.967542      0.14065       0.475746
  0.387567    0.27039    -0.223944    1.01172   …   0.341578      0.267425
  0.0540531   0.38316    -0.452211    0.706943      0.010038      0.476519
 -0.0318675   0.837272   -0.236756   -0.310497      0.180282      0.445651
  0.0200429  -1.47475     0.4939      0.500315     -0.122612      0.233625
  0.066215   -1.72938    -0.169337    0.17992      -0.000744307  -0.115629
  0.166773   -1.86715    -0.466514   -0.228327  …   0.13         -0.171712
  0.0883901  -1.03962     0.480752    0.390763      0.0648888     0.127738
 -0.0419525   2.38272     1.47009     0.687139      0.0411181     0.15320

In [14]:
patients = load_patients();

In [17]:
argmin([p["num_observations"] for p in patients])

3292

In [18]:
p = patients[3292]

Dict{String,Any} with 8 entries:
  "num_observations" => 9
  "mortality"        => true
  "patient_index"    => 3292
  "actions"          => Any[25, 22, 22, 22, 22, 1, 1, 1, 1]
  "icuid"            => 261511
  "MAP"              => Any[66.2143, 96.25, 92.3333, 95, 95, 95, 95, 95, 75]
  "cont_actions"     => Any[Any[0.8445, 0.7315, 0.731, 0.45, 0.45, 0, 0, 0, 0],…
  "trajectory"       => Any[Any[0.444369, -0.608163, -0.457158, -0.565437, -0.8…

In [19]:
p["trajectory"]

9-element Array{Any,1}:
 Any[0.4443686888037049, -0.6081628889391005, -0.457157771718115, -0.5654368132942825, -0.8517606244184172, -0.4284065758648318, -0.757032951617568, -0.3104969812368465, 0.31830388348918104, -0.21094937102428715  …  0.20108823634888146, -0.20929600158369863, -0.21078066220323344, -0.2120758299638234, 1.5276727454518275, 1.5303530318171399, 1.5330857462285579, 0.872991438835733, 0.8729867770096593, 0.8707857446426512]
 Any[0.4443686888037049, -0.6081628889391005, -0.457157771718115, -0.5654368132942825, -0.8517606244184172, -0.4284065758648318, -0.757032951617568, -0.3104969812368465, -0.02503459399529127, 0.6062318965347004  …  -0.47550127393045616, -0.20929600158369863, -0.21078066220323344, -0.2120758299638234, 1.5276727454518275, 1.5303530318171399, 1.5330857462285579, 0.872991438835733, 0.8729867770096593, 0.8707857446426512]
 Any[0.4443686888037049, -0.6081628889391005, -0.457157771718115, -0.5654368132942825, -0.8517606244184172, -0.4284065758648318, -0.75

In [20]:
p3292 = nothing

for pid = 1:n_patients
    p       = patients[pid]
    states  = []
    rewards = []
    tr      = p["trajectory"]
    tr_len  = size(tr)[1] 
    dims    = size(tr[1])[1]

    for j  = 1:tr_len
        d    = Float64(Inf)
        idx  = -1
        tr_k = convert(Array{Float64, 1}, tr[j])
        for k = 1:n_states
            d_k = distance(tr_k, C[:, k], dims)
            if d_k < d
                d   = d_k
                idx = k
            end
        end
        push!(states, idx)
        push!(rewards, 0.0)
    end

    if p["mortality"]
        push!(states, STATE_DECEASED)
        rewards[end] = -1.0
    else
        push!(states, STATE_SURVIVED)
        rewards[end] = 1.0
    end    

    mdp_p = Dict()
    mdp_p["trajectory"]      = states
    mdp_p["rewards"]         = rewards
    mdp_p["actions"]         = p["actions"]
    mdp_p["cont_actions"]    = p["cont_actions"]
    mdp_p["cont_trajectory"] = p["trajectory"]
    
    if pid == 3292
        p3292 = mdp_p
    end

    json_mdp_p = JSON.json(mdp_p)
    filename   = apath * "mdp_patient_" * string(pid) * ".json"

    open(filename, "w") do io
       println(io, json_mdp_p)
    end
end


In [21]:
p3292

Dict{Any,Any} with 5 entries:
  "actions"         => Any[25, 22, 22, 22, 22, 1, 1, 1, 1]
  "cont_trajectory" => Any[Any[0.444369, -0.608163, -0.457158, -0.565437, -0.85…
  "rewards"         => Any[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0]
  "cont_actions"    => Any[Any[0.8445, 0.7315, 0.731, 0.45, 0.45, 0, 0, 0, 0], …
  "trajectory"      => Any[3, 30, 301, 301, 626, 626, 245, 245, 358, 751]